# Jittikan Narapan ID: 6410422019

## Import Libraries and Datasets

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.rc("font", size=14) #dictionary objects
pylab.rcParams['figure.figsize'] = 16,16
%matplotlib inline

import seaborn as sns
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)

In [ ]:
#Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import RFECV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from mlxtend.preprocessing import minmax_scaling
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

import warnings
warnings.simplefilter(action='ignore')

In [ ]:
#Load Datasets
df_train = pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
df_test = pd.read_csv('/kaggle/input/dads6003-in-class-competition/test.csv')

In [ ]:
#Preview Training Data
print(df_train.info())
print(df_train.head())

There are null values in feature Xs (Except for x9) in Training dataset.

In [ ]:
#Preview Testing Data
print(df_test.info())
print(df_test.head())

There are no null values in Testing dataset.

In [ ]:
#Check the number of samples
print('The number of samples in the training dataset is {}.'.format(df_train.shape[0]))
print('The number of samples in the test dataset is {}.'.format(df_test.shape[0]))

## Create Function

In [ ]:
#Missing Values Data Analysis

def missingdata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    f,ax =plt.subplots(figsize=(8,6))
    plt.xticks(rotation='90')
    fig=sns.barplot(ms.index, ms["Percent"],color="purple",alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('%Missing values', fontsize=15)
    plt.title('Percentage of Null values by features', fontsize=15)
    return ms

## 1. Cleansing Data

### 1.1 Missing/Null Values

In [ ]:
#Check missing values
missingdata(df_train)

As mentioned earlier that only x9 contains no null value while there are less than 2% of missing values in other features. Therefore, We can use all 20 features in the next step.

### 1.2 Data Statistical Summary

In [ ]:
#Data Description
df_train.describe().T

We can see that the ranges of data in each feature are much different --> Need scaling.

### 1.3 Check Outliers

In [ ]:
f, axes = plt.subplots(ncols=4,nrows=5, figsize=(25,25))
data=df_train

sns.boxplot(x='y', y='x1', data=data, palette='autumn', ax=axes[0,0])
sns.boxplot(x='y', y='x2', data=data, palette='autumn', ax=axes[0,1])
sns.boxplot(x='y', y='x3', data=data, palette='autumn', ax=axes[0,2])
sns.boxplot(x='y', y='x4', data=data, palette='autumn', ax=axes[0,3])

sns.boxplot(x='y', y='x5', data=data, palette='autumn', ax=axes[1,0])
sns.boxplot(x='y', y='x6', data=data, palette='autumn', ax=axes[1,1])
sns.boxplot(x='y', y='x7', data=data, palette='autumn', ax=axes[1,2])
sns.boxplot(x='y', y='x8', data=data, palette='autumn', ax=axes[1,3])

sns.boxplot(x='y', y='x9', data=data, palette='autumn', ax=axes[2,0])
sns.boxplot(x='y', y='x10', data=data, palette='autumn', ax=axes[2,1])
sns.boxplot(x='y', y='x11', data=data, palette='autumn', ax=axes[2,2])
sns.boxplot(x='y', y='x12', data=data, palette='autumn', ax=axes[2,3])

sns.boxplot(x='y', y='x13', data=data, palette='autumn', ax=axes[3,0])
sns.boxplot(x='y', y='x14', data=data, palette='autumn', ax=axes[3,1])
sns.boxplot(x='y', y='x15', data=data, palette='autumn', ax=axes[3,2])
sns.boxplot(x='y', y='x16', data=data, palette='autumn', ax=axes[3,3])

sns.boxplot(x='y', y='x17', data=data, palette='autumn', ax=axes[4,0])
sns.boxplot(x='y', y='x18', data=data, palette='autumn', ax=axes[4,1])
sns.boxplot(x='y', y='x19', data=data, palette='autumn', ax=axes[4,2])
sns.boxplot(x='y', y='x20', data=data, palette='autumn', ax=axes[4,3])
plt.show()

From the box plots, we can see that there are some outliers in each features.

In [ ]:
#Remove Extreme Outliers

Q1 = df_train.quantile(0.25)
Q3 = df_train.quantile(0.75)
IQR = Q3 - Q1
df1 = df_train[~((df_train < (Q1 - 1.5 * IQR)) |(df_train > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df1.shape)
print(df1.head())

In [ ]:
#Replot 
f, axes = plt.subplots(ncols=4,nrows=5, figsize=(25,25))
data=df1

sns.boxplot(x='y', y='x1', data=data, palette='magma', ax=axes[0,0])
sns.boxplot(x='y', y='x2', data=data, palette='magma', ax=axes[0,1])
sns.boxplot(x='y', y='x3', data=data, palette='magma', ax=axes[0,2])
sns.boxplot(x='y', y='x4', data=data, palette='magma', ax=axes[0,3])

sns.boxplot(x='y', y='x5', data=data, palette='magma', ax=axes[1,0])
sns.boxplot(x='y', y='x6', data=data, palette='magma', ax=axes[1,1])
sns.boxplot(x='y', y='x7', data=data, palette='magma', ax=axes[1,2])
sns.boxplot(x='y', y='x8', data=data, palette='magma', ax=axes[1,3])

sns.boxplot(x='y', y='x9', data=data, palette='magma', ax=axes[2,0])
sns.boxplot(x='y', y='x10', data=data, palette='magma', ax=axes[2,1])
sns.boxplot(x='y', y='x11', data=data, palette='magma', ax=axes[2,2])
sns.boxplot(x='y', y='x12', data=data, palette='magma', ax=axes[2,3])

sns.boxplot(x='y', y='x13', data=data, palette='magma', ax=axes[3,0])
sns.boxplot(x='y', y='x14', data=data, palette='magma', ax=axes[3,1])
sns.boxplot(x='y', y='x15', data=data, palette='magma', ax=axes[3,2])
sns.boxplot(x='y', y='x16', data=data, palette='magma', ax=axes[3,3])

sns.boxplot(x='y', y='x17', data=data, palette='magma', ax=axes[4,0])
sns.boxplot(x='y', y='x18', data=data, palette='magma', ax=axes[4,1])
sns.boxplot(x='y', y='x19', data=data, palette='magma', ax=axes[4,2])
sns.boxplot(x='y', y='x20', data=data, palette='magma', ax=axes[4,3])
plt.show()

From box plots above, extreme outliers are removed.

### 1.4 Data Distribution

In [ ]:
#Check if in each binary results in 'y', means of each feature are different or not.
df=df1.groupby(['y']).mean()
print(df)
print(df.pct_change()*100)

Difference of means are significantly noticed in features x2, x3, x9, and x15

In [ ]:
#Check distribution of features data grouped by y values
ax = df1.groupby(['y']).hist(bins=10, figsize=(18,20))
plt.show()

* y = 0: Means of each feature show normal distributions.
* y = 1: Most of the features show normal distributions but some features show skewed/slightly skewed distributions (x1,x2, x3, x14, and x15).
* Therefore, We will replace Null values in each features with Median values.

### 1.5 Fill Null Values

In [ ]:
#Fill missing values with mean grouped by y

for i in df1:
    df1[i] = df1[i].fillna(df1.groupby('y')[i].transform('median'))

#Check if we fill all the Null values
df1.isnull().sum()

Now, we have data ready for further EDA.

## 2. Exploratory Data Analysis (describe insight and visualization)

In [ ]:
#Percentage of y results in dataset - Check if data is imbalanced or not
df1['y'].value_counts()
ax = sns.countplot(x='y',data=df1, palette='hls')
ax.bar_label(ax.containers[0])
print('y: 0', round(df1['y'].value_counts()[0]/len(df1) * 100,2), '% of the dataset')
print('y: 1', round(df1['y'].value_counts()[1]/len(df1) * 100,2), '% of the dataset')

This dataset is considered as imbalanced with approximated ratio of y=1 to y=0 of 1/4.

In [ ]:
#Violin plot to see each feature distributions to y value
df_train_scale = minmax_scaling(df_train, df_train.columns, 1)
df2 = pd.melt(df_train_scale,id_vars='y',var_name = 'features',value_name='value')
plt.figure(figsize=(15,10))
sns.violinplot(x='features',y='value',hue='y',data=df2,split=True,inner='quart')
plt.xticks(rotation=90)

Features x2,x3,x9, and x15 show significant differences in means grouped by y-value.

In [ ]:
#Identify Correlated Variables - Correlation Map

f,axes = plt.subplots(ncols=2,figsize=(24,10))

sns.heatmap(df_train.corr(), annot=True, linewidths=.5,fmt='.1f',ax=axes[0])
axes[0].set_title('Training Data Features Correlation with Y')
sns.heatmap(df_test.corr(), annot=True, linewidths=.5,fmt='.1f',ax=axes[1])
axes[1].set_title('Testing Data Features Correlation')

plt.show()

* Training dataset shows that y value correlates to main 4 features: x2,x3,x9, and x15 with approximated correlation efficient -0.4 to -0.5.
* These 4 features also show correlations with x6,x7,x8,x10,x12, and x14 --- Similar correlations among features to Testing dataset

## 3. Training Data - Cross Validate

### 3.1 Split Data and Scaling Data
* Using StratifiedKFold for splitting data as we've seen imbalanced data in our training dataset to ensure the similar proportion are splitted into the training and validating datasets
* Scaling data after splitting as we've seen the different range of data in each feature Xs

In [ ]:
#Split and Scaling Training and Testing (Validating) Data in Training File using train 80% and test 20%

df1_x = ['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','x13','x14','x15','x16','x17','x18','x19','x20'] 
X = df1[df1_x]
y = df1['y']

skf = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)

#train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.20,random_state=0) (Hold-out Method)

for train_index, test_index in skf.split(X,y):
    train_x, test_x = X.iloc[train_index], X.iloc[test_index]
    train_y, test_y = y.iloc[train_index], y.iloc[test_index]

#Scale data & Define Scaler
scaler = MinMaxScaler()
scaler.fit(train_x)

train_x_norm = pd.DataFrame(scaler.transform(train_x))        #Scaling training dataset
test_x_norm = pd.DataFrame(scaler.transform(test_x))          #Scaling validating dataset
df_test_norm = pd.DataFrame(scaler.transform(df_test))        #Scaling testing dataset

train_x_norm.columns = df1_x
test_x_norm.columns = df1_x
df_test_norm.columns = df1_x

df_train_norm = pd.concat((train_x_norm, train_y), 1)
df_val_norm = pd.concat((test_x_norm,test_y),1)

print('df1 Shape: {}'.format(df1.shape))
print('Train Shape: {}'.format(train_x.shape))
print('Validation(Test) Shape: {}'.format(test_x.shape))

print(train_y.value_counts())
print('y: 0', round(train_y.value_counts()[0]/len(train_y) * 100,2), '% of the dataset')
print('y: 1', round(train_y.value_counts()[1]/len(train_y) * 100,2), '% of the dataset')

In [ ]:
#Correlation Map

f,axes = plt.subplots(ncols=3,figsize=(36,12))

sns.heatmap(train_x_norm.corr(), annot=True, linewidths=.5,fmt='.1f',ax=axes[0])
axes[0].set_title('Training Data Correlation')
sns.heatmap(test_x_norm.corr(), annot=True, linewidths=.5,fmt='.1f',ax=axes[1])
axes[1].set_title('Validating Data Correlation')
sns.heatmap(df_test_norm.corr(), annot=True, linewidths=.5,fmt='.1f',ax=axes[2])
axes[2].set_title('Testing Data Correlation')
plt.show()

After Scaling datasets, training, validating, and testing datasets show the same correlation maps with previous relationships.

### 3.2 Build ML Models: Using all 20 features

In [ ]:
#Spot Check Algorithms
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('KNearestNeighbors', KNeighborsClassifier()))
models.append(('DecisionTree', DecisionTreeClassifier()))
models.append(('SupportVectorMachines', SVC()))
models.append(('RandomForest', RandomForestClassifier()))
#Evaluate training score for each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
    cv_results = cross_val_score(model, train_x_norm, train_y, cv=kfold, scoring='f1_weighted')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
#Compare Algorithms
plt.figure(figsize=(10,6))
plt.boxplot(results, labels=names)
plt.title('Algorithms Comparison')
plt.show()

Preliminary, we can choose top two algorithms to model the datasets: SupportVectorMachines and RandomForest



### 3.3 Feature Selections

In [ ]:
#Features Selection - Recursive Feature Elimination with Cross Validation and Random Forest

clf_rf = RandomForestClassifier()
rfecv = RFECV(estimator=clf_rf, step=1, cv=10, scoring='f1_weighted') #10-fold cross-validation
rfecv = rfecv.fit(train_x_norm,train_y)

print('Optimal number of features: ', rfecv.n_features_)
print('Best Features: ',train_x_norm.columns[rfecv.support_])

# Plot number of features VS. cross-validation scores
plt.figure(figsize=(10,6))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
#Selected Features
selfeat = ['x2', 'x3', 'x4','x6', 'x7', 'x8', 'x9', 'x10', 'x12', 'x14', 'x15']

In [ ]:
#Spot Check Algorithms for Selected Features
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('KNearestNeighbors', KNeighborsClassifier()))
models.append(('DecisionTree', DecisionTreeClassifier()))
models.append(('SupportVectorMachines', SVC()))
models.append(('RandomForest', RandomForestClassifier()))
#Evaluate training score for each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
    cv_results = cross_val_score(model, train_x_norm[selfeat], train_y, cv=kfold, scoring='f1_weighted')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
#Compare Algorithms for Selected Features
plt.figure(figsize=(10,6))
plt.boxplot(results, labels=names)
plt.title('Algorithms Comparison')
plt.show()

After features selection, we can choose top three algorithms to model the datasets: SupportVectorMachines, KNearestNeighbors, and RandomForest

## 4. Testing Data - with Validating dataset
* Test Validating datasets with top three algorithms: SupportVectorMachines, KNearestNeighbors, and RandomForest

In [ ]:
#SupportVectorMachines
svc = SVC(gamma='auto')
svc.fit(train_x_norm[selfeat], train_y)
y_pred_svc = svc.predict(test_x_norm[selfeat])
svc_cf = confusion_matrix(test_y,y_pred_svc)
print('SupportVectorMachines:')
print(classification_report(test_y, y_pred_svc))
f1svc = '{0:.4g}'.format(f1_score(test_y, y_pred_svc, average='weighted'))
accsvc = '{0:.4g}'.format(accuracy_score(test_y, y_pred_svc))
print('SVC F1 Score: ',f1svc)
print('SVC Accuracy Score: ',accsvc)

#KNearestNeighbors
knn = KNeighborsClassifier()
knn.fit(train_x_norm[selfeat], train_y)
y_pred_knn = knn.predict(test_x_norm[selfeat])
knn_cf = confusion_matrix(test_y,y_pred_knn)
print('KNearestNeighbors:')
print(classification_report(test_y, y_pred_knn))
f1knn = '{0:.4g}'.format(f1_score(test_y, y_pred_knn, average='weighted'))
accknn = '{0:.4g}'.format(accuracy_score(test_y, y_pred_knn))
print('KNearestNeighbors F1 Score: ',f1knn)
print('KNearestNeighbors Accuracy Score: ',accknn)

#RandomForest
rf = RandomForestClassifier()
rf.fit(train_x_norm[selfeat], train_y)
y_pred_rf = rf.predict(test_x_norm[selfeat])
rf_cf = confusion_matrix(test_y,y_pred_rf)
print('RandomForest:')
print(classification_report(test_y, y_pred_rf))
f1rf = '{0:.4g}'.format(f1_score(test_y, y_pred_rf, average='weighted'))
accrf = '{0:.4g}'.format(accuracy_score(test_y, y_pred_rf))
print('RandomForest F1 Score: ',f1rf)
print('RandomForest Accuracy Score: ',accrf)

#Plot Confusion Metrixes
f,axes = plt.subplots(ncols=3,figsize=(36,12))
sns.heatmap(svc_cf, annot=True,fmt='2.0f',cmap='summer',ax=axes[0])
axes[0].set_title('SupportVectorMachines \n Confusion Matrix')

sns.heatmap(knn_cf, annot=True,fmt='2.0f',cmap='summer',ax=axes[1])
axes[1].set_title('KNearestNeighbors \n Confusion Matrix')

sns.heatmap(rf_cf, annot=True,fmt='2.0f',cmap='summer',ax=axes[2])
axes[2].set_title('RandomForest \n Confusion Matrix')
plt.show()

### 4.1 Check Cross Validation Score

In [ ]:
#SVC
#Define the pipeline
step1 = list()
step1.append(('scaler', MinMaxScaler()))
step1.append(('model', SVC()))
pipeline1 = Pipeline(steps=step1)
#Define the evaluation procedure
cv = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
#Evaluate the model using cross-validation
scores1 = cross_val_score(pipeline1, X[selfeat], y, scoring='f1_weighted', cv=cv, n_jobs=-1)
#Report performance
print('SVC Accuracy: %.3f (%.3f)' % (scores1.mean()*100, scores1.std()*100))

#KNearestNeighbors
#Define the pipeline
step2 = list()
step2.append(('scaler', MinMaxScaler()))
step2.append(('model', KNeighborsClassifier()))
pipeline2 = Pipeline(steps=step2)
#Define the evaluation procedure
cv = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
#Evaluate the model using cross-validation
scores2 = cross_val_score(pipeline2, X[selfeat], y, scoring='f1_weighted', cv=cv, n_jobs=-1)
#Report performance
print('KNearestNeighbors Accuracy: %.3f (%.3f)' % (scores2.mean()*100, scores2.std()*100))


#RandomForest
#Define the pipeline
step3 = list()
step3.append(('scaler', MinMaxScaler()))
step3.append(('model', RandomForestClassifier()))
pipeline3 = Pipeline(steps=step3)
#Define the evaluation procedure
cv = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)
#Evaluate the model using cross-validation
scores3 = cross_val_score(pipeline3, X[selfeat], y, scoring='f1_weighted', cv=cv, n_jobs=-1)
#Report performance
print('RandomForest Accuracy: %.3f (%.3f)' % (scores3.mean()*100, scores3.std()*100))

### 4.2 Tuning HyperParameters using GridSearch

In [ ]:
#SupportVectorMachines
svc_params = {'C': [0.5, 0.6, 0.7,0.8, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
grid_svc = GridSearchCV(SVC(), svc_params, cv=5, scoring='f1_weighted', n_jobs= 4, verbose = 1)
grid_svc.fit(train_x_norm[selfeat], train_y)
svm_svc = grid_svc.best_params_
print(svm_svc)


In [ ]:
#KNN
knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}
grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params, cv=5, scoring='f1_weighted', n_jobs= 4, verbose = 1)
grid_knears.fit(train_x_norm[selfeat], train_y)
knears_neighbors = grid_knears.best_estimator_
print(knears_neighbors)

In [ ]:
#RandomForest
rf_params = {'n_estimators': range(100,1000,100),'max_features': ['auto', 'sqrt'],'bootstrap': [True,False]}
grid_rf = GridSearchCV(RandomForestClassifier(), rf_params, cv=5, scoring='f1_weighted', n_jobs= 4, verbose = 1)
grid_rf.fit(train_x_norm[selfeat], train_y)
rf_rf = grid_rf.best_estimator_
print(rf_rf)


### 4.3 Apply HyperParameters

In [ ]:
#SupportVectorMachines
svmsvc = SVC(C=0.9, kernel='poly')
svmsvc.fit(train_x_norm[selfeat],train_y)

y_pred_svmsvc = svmsvc.predict(test_x_norm[selfeat])
svmsvc_cf = confusion_matrix(test_y,y_pred_svmsvc)

ax = sns.heatmap(svmsvc_cf, annot=True,fmt='2.0f',cmap='summer')
ax.set_title('SupportVectorMachines_Tuned \n Confusion Matrix')

plt.show()

print('SupportVectorMachines_Tuned:')
print(classification_report(test_y, y_pred_svmsvc))
f1svmsvc = f1_score(test_y, y_pred_svmsvc, average='weighted')
svmsvc_cv_score = cross_val_score(svmsvc, train_x_norm[selfeat], train_y, cv=10,scoring='f1_weighted')
print(f'Cross Validation Score: {svmsvc_cv_score.mean():.4f}')
print(f'F1 Score: {f1svmsvc:.4f}')

In [ ]:
#KNN
knearneigh = KNeighborsClassifier(n_neighbors=3)
knearneigh.fit(train_x_norm[selfeat],train_y)

y_pred_knearneigh = knearneigh.predict(test_x_norm[selfeat])
knearneigh_cf = confusion_matrix(test_y,y_pred_knearneigh)

ax = sns.heatmap(knearneigh_cf, annot=True,fmt='2.0f',cmap='summer')
ax.set_title('KNearestNeighbors_Tuned \n Confusion Matrix')

plt.show()

print('KNearestNeighbors_Tuned:')
print(classification_report(test_y, y_pred_knearneigh))
f1knearneigh = f1_score(test_y, y_pred_knearneigh, average='weighted')
knearneigh_cv_score = cross_val_score(knearneigh, train_x_norm[selfeat], train_y, cv=10,scoring='f1_weighted')
print(f'{knearneigh_cv_score.mean():.4f}')
print(f'{f1knearneigh:.4f}')

In [ ]:
#RandomForest
randomforest = RandomForestClassifier(bootstrap=False, max_features='sqrt', n_estimators=900)
randomforest.fit(train_x_norm[selfeat],train_y)

y_pred_randomforest = randomforest.predict(test_x_norm[selfeat])
randomforest_cf = confusion_matrix(test_y,y_pred_randomforest)

ax = sns.heatmap(randomforest_cf, annot=True,fmt='2.0f',cmap='summer')
ax.set_title('RandomForest_Tuned \n Confusion Matrix')

plt.show()

print('RandomForest_Tuned:')
print(classification_report(test_y, y_pred_randomforest))
f1randomforest = f1_score(test_y, y_pred_randomforest, average='weighted')
rf_cv_score = cross_val_score(randomforest, train_x_norm[selfeat], train_y, cv=10,scoring='f1_weighted')
print(f'{rf_cv_score.mean():.4f}')
print(f'{f1randomforest:.4f}')

In [4]:
#Check Feature Importance
print("Important features")
pd.Series(randomforest.feature_importances_,train_x_norm[selfeat].columns).sort_values(ascending=True).plot.barh(width=0.8)
print('__'*30)

## Create Output for Submission
### Model Selected: Support Vector Machines with tuning hyperparameters (C=0.9, kernel='poly')

In [ ]:
df_test_norm['y'] = svmsvc.predict(df_test_norm[selfeat])
df_test_norm.head()

df_test_norm['y'].value_counts()
ax = sns.countplot(x='y',data=df_test_norm, palette='hls')
ax.bar_label(ax.containers[0])

In [ ]:
#Output
predict = df_test_norm['y']
fields = ['id','Expected']
df = pd.DataFrame()
id = list(range(1,2501))
df['id']=pd.Series(id)
df['Expected']=pd.Series(predict)
df.to_csv('submit_svmsvc2.csv', index=False)

------------------------------------------ END -----------------------------------------------------